In [1]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import string

from toydown import GeoUnit, ToyDown

In [2]:
def create_demo_tree_homogenous_branching(h=3):
    """ Creates a demo tree of height h <= 16, with homogenous branching of h.
        This demo tree has two population counts, and count that is the sum of them.
    """
    leaves = ["1" + "".join(a) for a in itertools.product(string.hexdigits[1:h+1], repeat=h-1)]
    leaves_counts = np.random.randint(1000, size=(len(leaves),2))
    leaves_attrs = np.insert(leaves_counts, 0, leaves_counts.sum(axis=1), axis=1)
    
    nodes = {i: j for i, j in zip(leaves, leaves_attrs)}
    
    for i in range(2, h+1):
        level_names = list(set(list(map(lambda s: s[:-(i-1)], leaves))))
        level_counts = [np.zeros(3)]*len(level_names)
        for node in level_names:
            nodes[node] = np.array([v for k, v in nodes.items() if k.startswith(node) and len(k) == h]).sum(axis=0)
        
    return [GeoUnit(k, k[:-1], v) if k != "1" else GeoUnit(k, None, v) for k, v in nodes.items()]

In [10]:
geounits = create_demo_tree_homogenous_branching(3)
geounits.reverse()
eps = 2
eps_split = [0.25, 0.25, 0.5]

In [11]:
model = ToyDown(geounits, 3, eps, eps_split)
model.show()

1
├── 11
│   ├── 111
│   ├── 112
│   └── 113
├── 12
│   ├── 121
│   ├── 122
│   └── 123
└── 13
    ├── 131
    ├── 132
    └── 133



In [12]:
## Constraints: 0 difference among partition bins
## number of children n, returns list of 0 diff constraints.
cons_0_diff = lambda n: [{'type': 'eq', 'fun': lambda x, i=i:  x[i] - np.sum([x[j] for j in range(i+1,i+3)])} 
                         for i in np.arange(n*3, step=3)]

In [13]:
model.noise_and_adjust(node_cons=cons_0_diff, opts={"maxiter": 500, "disp": True})

1
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9.057239362788096
            Iterations: 2
            Function evaluations: 10
            Gradient evaluations: 2
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 13.761661068718468
            Iterations: 40
            Function evaluations: 467
            Gradient evaluations: 40
11
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.2839598258068605
            Iterations: 236
            Function evaluations: 3948
            Gradient evaluations: 236
113
112
111
12
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.0178999974572704
            Iterations: 44
            Function evaluations: 526
            Gradient evaluations: 44
123
122
121
13
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 7.450452786251241
         

In [14]:
model.get_node("1").data.__dict__

{'name': '1',
 'parent': None,
 'attributes': array([8550, 4354, 4196]),
 'identifier': '1',
 'level': 0,
 'noised': array([8540.97806797, 4354.69281847, 4195.34248886]),
 'noise': array([-9.02193203,  0.69281847, -0.65751114]),
 'noise_type': 'laplacian',
 'adjusted': array([8549.33333333, 4354.66666667, 4194.66666667]),
 'error': array([ 0.66666667, -0.66666667,  1.33333333])}

In [15]:
model.get_node("12").data.__dict__

{'name': '12',
 'parent': '1',
 'attributes': array([2698, 1339, 1359]),
 'identifier': '12',
 'level': 1,
 'noised': array([2698.67655158, 1343.40009759, 1358.60379522]),
 'noise': array([ 0.67655158,  4.40009759, -0.39620478]),
 'noise_type': 'laplacian',
 'adjusted': array([2698.46333036, 1339.81213398, 1358.65119638]),
 'error': array([-0.46333036, -0.81213398,  0.34880362])}

In [16]:
model.get_node("123").data.__dict__

{'name': '123',
 'parent': '12',
 'attributes': array([593, 539,  54]),
 'identifier': '123',
 'level': 2,
 'noised': array([594.62890329, 539.38827436,  56.00672072]),
 'noise': array([1.62890329, 0.38827436, 2.00672072]),
 'noise_type': 'laplacian',
 'adjusted': array([594.44858491, 539.54389202,  54.90469289]),
 'error': array([-1.44858491, -0.54389202, -0.90469289])}